<div style="padding: 10px; border: 2px solid green; background-color: #ccffcc; color: darkgreen; font-size: 16px;">
  <strong>Achtung:</strong> Wenn Sie die Bearbeitung dieser Aufgabe erneut aufnehmen, dann führen Sie bitte immer alle Codezellen, angefangen bei 'Bibliotheken (Libraries) importieren', erneut aus!
</div>

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from database import Database
import pandas as pd
import plotly.express as px

In [ ]:
class ETL:
    final_table = pd.DataFrame()

    def __init__(self, database: Database = None):
        self.database = database
        self.tables = None
        self.df_final_table = None

    def extract(self):
        self.database.load()
        self.tables = self.database.tables

    def transform(self):
        df_vehicle_details = pd.merge(self.tables["h_vehicle"],
                                      self.tables["l_vehicle_sales_code"],
                                      left_on="h_vehicle_hash",
                                      right_on="h_vehicle_hash",
                                      how="left")
        df_vehicle_details = pd.merge(df_vehicle_details,
                                      self.tables["h_sales_code"],
                                      left_on="h_sales_code_hash",
                                      right_on="h_sales_code_hash",
                                      how="left")
        df_vehicle_details = pd.merge(df_vehicle_details,
                                      self.tables["s_vehicle_axle_details"],
                                      left_on="h_vehicle_hash",
                                      right_on="h_vehicle_hash",
                                      how="left")
        df_vehicle_details = pd.merge(df_vehicle_details,
                                      self.tables["vedoc_asa"],
                                      left_on="fin",
                                      right_on="fin",
                                      how="left")
        self.df_final_table = df_vehicle_details.drop(columns=["h_vehicle_hash", "h_sales_code_hash"])
        print("transformation completed")

    def load(self):
        pass

    def run(self):
        self.extract()
        self.clean_raw_data()
        self.transform()
        self.load()
        return self.df_final_table

    def clean_raw_data(self):
        self.drop_unnamed_columns()
        self.remove_invalid_fins()

    def drop_unnamed_columns(self):
        for table_name, table in self.tables.items():
            self.tables[table_name] = self.tables[table_name].drop(columns=["Unnamed: 0"])

    def remove_invalid_fins(self):
        df = self.tables["h_vehicle"]
        df = df[df["fin"].astype(str).map(len) == 17]
        self.tables["h_vehicle"] = df

In [ ]:
class VehicleDetailsAnalyzer:
    def __init__(self, df_vehicle_details: pd.DataFrame = None):
        self.countries = {"ger": 254, "fr": 561, "es": 649, "uk": 268, "us": 768}
        self.df_vehicle_details = df_vehicle_details
        self.df_vehicle_unique = None

    def run(self):
        self.preprocess_data()
        self.visualize()

    def preprocess_data(self):
        self.df_vehicle_details["prod_dt"] = pd.to_datetime(self.df_vehicle_details["prod_dt"])
        self.df_vehicle_details["prod_year"] = self.df_vehicle_details["prod_dt"].dt.year
        self.df_vehicle_details["prod_month"] = self.df_vehicle_details["prod_dt"].dt.month
        self.df_vehicle_unique = self.df_vehicle_details.drop_duplicates("fin")

    def visualize(self):
        self.plot_production_number_over_years()
        self.plot_production_number_over_years_per_sales_area()
        self.plot_engine_over_sales_area()

    def plot_engine_over_sales_area(self):
        df = self.df_vehicle_details
        df = df[df["sales_code"] == "Z5E"]
        df = df.groupby("sales_area_nm").count()
        df = df[df.index.isin(self.countries.values())]
        df = df.rename(index={value: key for key, value in self.countries.items()})
        px.pie(data_frame=df, names=df.index, values="fin",
               title="production with OM471 over sales countries").show()

    def plot_production_number_over_years_per_sales_area(self):
        df = self.df_vehicle_unique
        df_final = pd.DataFrame()
        for country, area_nm in self.countries.items():
            df_tmp = df[df["sales_area_nm"] == area_nm]
            df_final[country] = df_tmp.groupby("prod_year").count()["fin"]
        fig = (px.bar(data_frame=df_final, x=df_final.index, y=df_final.columns, barmode='group',
                      title="production over year per sales country",
                      labels={"value": "number of production",
                              "prod_year": "production year",
                              "variable": "country"}))
        fig.update_yaxes(range=[5, None])
        fig.show()

    def plot_production_number_over_years(self):
        df = self.df_vehicle_unique
        df = df[(df['prod_year'] >= 2015) & (df['prod_year'] <= 2022)]
        df = df.groupby("prod_year").count()
        fig = px.bar(data_frame=df, x=df.index, y="fin",
                     title="production over year",
                     labels={"fin": "number of production",
                             "prod_year": "production year"})
        fig.update_yaxes(range=[3500, 4000])
        fig.show()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()